In [ ]:
import numpy as np
import os
from components.network import Helper, Network

positives = Helper.get_positives(number_of_samples=500)
filepath = os.path.join(os.path.pardir, 'data/train_test_file_small.npz')
np.savez(filepath, positives=positives)

In [ ]:
import os
import numpy as np
import librosa
from components.network import Helper

filepath = os.path.join(os.path.pardir, 'data/train_test_file_small.npz')
file = np.load(filepath, allow_pickle=True)
positives = file['positives']
negatives = Helper.get_negatives(500, sample_length=660719)
train, test = Helper.prepare_dataset(positives, negatives, train_to_test_ratio=0.8)

train_mel = Helper.convert_to_feature(train, librosa.feature.melspectrogram)
test_mel = Helper.convert_to_feature(test, librosa.feature.melspectrogram)
x_train_mel, y_train_mel = Helper.get_x_y(train_mel)
x_test_mel, y_test_mel = Helper.get_x_y(test_mel)

train_tempo = Helper.convert_to_feature(train, librosa.feature.tempogram)
test_tempo = Helper.convert_to_feature(test, librosa.feature.tempogram)
x_train_tempo, y_train_tempo = Helper.get_x_y(train_tempo)
x_test_tempo, y_test_tempo = Helper.get_x_y(test_tempo)

train_rms = Helper.convert_to_feature(train, librosa.feature.rms)
test_rms = Helper.convert_to_feature(test, librosa.feature.rms)
x_train_rms, y_train_rms = Helper.get_x_y(train_rms)
x_test_rms, y_test_rms = Helper.get_x_y(test_rms)

train_mfcc = Helper.convert_to_feature(train, librosa.feature.mfcc)
test_mfcc = Helper.convert_to_feature(test, librosa.feature.mfcc)
x_train_mfcc, y_train_mfcc = Helper.get_x_y(train_mfcc)
x_test_mfcc, y_test_mfcc = Helper.get_x_y(test_mfcc)

train_chroma = Helper.convert_to_feature(train, librosa.feature.chroma_stft)
test_chroma = Helper.convert_to_feature(test, librosa.feature.chroma_stft)
x_train_chroma, y_train_chroma = Helper.get_x_y(train_chroma)
x_test_chroma, y_test_chroma = Helper.get_x_y(test_chroma)


Number of all negative samples: 6965
Length of the specimen should be 660719


In [ ]:
from components.network import Network

network_mel = Network(input_shape=x_train_mel[0].shape)
network_tempo = Network(input_shape=x_train_tempo[0].shape)
network_rms = Network(input_shape=x_train_rms[0].shape)
network_mfcc = Network(input_shape=x_train_mfcc[0].shape)
network_chroma = Network(input_shape=x_train_chroma[0].shape)

In [1]:
import os
from components.network import Network

network_mel = Network(input_shape=(128, 1291))
network_tempo = Network(input_shape=(384, 1291))
network_rms = Network(input_shape=(1, 1291))
network_mfcc = Network(input_shape=(20, 1291))
network_chroma = Network(input_shape=(12, 1291))

network_mel.load_model(os.path.join(os.path.pardir, 'models/model_mel'))
network_tempo.load_model(os.path.join(os.path.pardir, 'models/model_tempo'))
network_rms.load_model(os.path.join(os.path.pardir, 'models/model_rms'))
network_mfcc.load_model(os.path.join(os.path.pardir, 'models/model_mfcc'))
network_chroma.load_model(os.path.join(os.path.pardir, 'models/model_chroma'))

Using plaidml.keras.backend backend.
INFO:plaidml:Opening device "opencl_amd_gfx1010.0"


In [ ]:
network_mel.train(x_train_mel, y_train_mel, epochs=40, batch_size=100)
network_tempo.train(x_train_tempo, y_train_tempo, epochs=40, batch_size=100)
network_rms.train(x_train_rms, y_train_rms, epochs=40, batch_size=100)
network_mfcc.train(x_train_mfcc, y_train_mfcc, epochs=40, batch_size=100)
network_chroma.train(x_train_chroma, y_train_chroma, epochs=40, batch_size=100)

network_mel.evaluate(x_test_mel, y_test_mel)
network_tempo.evaluate(x_test_tempo, y_test_tempo)
network_rms.evaluate(x_test_rms, y_test_rms)
network_mfcc.evaluate(x_test_mfcc, y_test_mfcc)
network_chroma.evaluate(x_test_chroma, y_test_chroma)

In [ ]:
import os

network_mel.save_model(os.path.join(os.path.pardir, f'models/model_mel'))
network_tempo.save_model(os.path.join(os.path.pardir, f'models/model_tempo'))
network_rms.save_model(os.path.join(os.path.pardir, f'models/model_rms'))
network_mfcc.save_model(os.path.join(os.path.pardir, f'models/model_mfcc'))
network_chroma.save_model(os.path.join(os.path.pardir, f'models/model_chroma'))

In [2]:
import librosa

from components.specimen import Evolution

#specimen_length = 660984
specimen_length = 660719
evolution = Evolution(neural_networks=[network_mel, network_tempo, network_rms, network_mfcc, network_chroma],
                      features_list=[librosa.feature.melspectrogram, librosa.feature.tempogram, librosa.feature.rms,
                                     librosa.feature.mfcc, librosa.feature.chroma_stft],
                      number_of_specimen=100,
                      specimen_length=specimen_length, target=None, mutation_chance=0.1, crossover_chance=0.1)
evolution.run_epochs(number_of_epochs=100, save=True, save_as_negative=False, epsilon=0.2)

Started fitness calculation --------------------
Started specimen conversion                    |
Finished specimen conversion                   v
Finished network 1/5 in 14.466999530792236s
Finished network 2/5 in 15.361998319625854s
Finished network 3/5 in 13.654000759124756s
Finished network 4/5 in 14.146999835968018s
Finished network 5/5 in 14.126000881195068s
Specimen got marks: [0.42952165, 0.0061067496, 0.998752, 1.0, 0.004443814]
Specimen got marks: [0.43338597, 0.0044600097, 0.9987538, 1.0, 0.0018101617]
Specimen got marks: [0.44725868, 0.0067640236, 0.9987476, 1.0, 0.02432321]
Specimen got marks: [0.43888065, 0.0065117762, 0.998746, 1.0, 0.0029298265]
Specimen got marks: [0.4417936, 0.005105832, 0.99874276, 1.0, 0.0024577563]
Specimen got marks: [0.44273752, 0.007899164, 0.99874514, 1.0, 0.0036719101]
Specimen got marks: [0.42880282, 0.0075954963, 0.9987477, 1.0, 0.0019098994]
Specimen got marks: [0.43765935, 0.007641598, 0.99875057, 1.0, 0.0017498852]
Specimen got marks: [0.

KeyboardInterrupt: 

In [ ]:
evolution.run_epochs(number_of_epochs=100, save=True, save_as_negative=False, epsilon=0.2)

